In [5]:
import math, random
import numpy as np
import cvxpy as cv
import matplotlib.pyplot as plt
import scipy

# Setup

- import packages
- define simulation of data for bounds

In [6]:
def sample_path(x0, params, tmax, plot=False):
    """
    Simulate sample path of birth death reaction

    x0 = initial state
    params = [k1, k2]
    tmax = time of simulation

    return states visited and jump times
    """

    # parameters
    k1 = params[0]
    k2 = params[1]

    # initialise time and state (also store jump times)
    t = 0
    path = [x0]
    jump_times = [0]

    # simulate until stopping time
    while t < tmax:

        # current state
        x = path[-1]

        # rates
        q_birth = k1
        q_death = k2 * x
        q_hold = q_birth + q_death

        # exponential holding time of state
        t_hold = -math.log(random.random()) / q_hold
        t += t_hold
        jump_times.append(t)

        # jump probability
        u = random.random()
        if u < q_birth / q_hold:
            path.append(x + 1)
        else:
            path.append(x - 1)

    if plot:
        plt.step(jump_times, path, where="post")

    return path, jump_times

In [7]:
def bootstrap(x_list, x0, params, tmax, n, N, plot=False, printing=False):
    """
    x_list = states to calculate probabilities for
    x0 = initial state
    params = [k1, k2]
    tmax = time until sample
    n = number of simulated samples
    N = number of bootstrap samples
    """

    # simulate n samples
    states = []
    for i in range(n):
        path, jumps = sample_path(x0, params, tmax, plot=plot)
        states.append(path[-2])
    if plot:
        plt.show()

    # simulate N bootstrap samples: estimates p(x) for each, and for each x
    estimates = [[] for x in x_list]
    for i in range(N):
        sample = random.choices(states,k = n)
        for i, x in enumerate(x_list):
            estimates[i].append(sample.count(x) / n)

    # create confidence intervals (95%) via 2.5%, 97.5% quantiles for each x
    intervals = [np.quantile(est,[0.025,0.975]) for est in estimates]
    
    # plot histograms and CI
    for i, x in enumerate(x_list):
        if printing:
            print(f"95% CI for p({x}) is: ({intervals[i][0]}, {intervals[i][1]})")
        if plot:
            plt.hist(estimates[i])
            plt.title(f"Hist of p({x})")
            plt.axvline(intervals[i][0], color="red")
            plt.axvline(intervals[i][1], color="red")
            plt.show()

    # return CIs
    return estimates, intervals

# Non-linear test

min/max k1

s.t. $ \sum Q^r k_r p = 0 $ 

$ p^l \le p \le p^u $

$ k _r \ge 0 $

But, without introducing $ z_r = k_r p $, the first constrant is non-linear, as multiply $ k_r $ and p, 2 variables, together. So this is no longer a linear program, and becomes a QCQP: quadratically constrained linear program

However, still have a linear objective and the non-linear terms are cross terms, somtimes called Bilinear program

### Problem: 

Not convex, as $ Q^r $ matrices are not symmetric, let alone PD or PSD, so have non-convex quadratically constrained problem.

### Approach 2:

use $ \sum Q^r z_r = 0 $ and define $ z_r = k_r p $ as a constraint

### But: still non-convex and non-linear constraint

In [17]:
# create confidence intervals
k1 = 2
x_max = 6
estimates, intervals = bootstrap([x for x in range(x_max)],0,[k1,1],100,1000,1000)

# create bounds on p
pl = [intv[0] for intv in intervals]
pu = [intv[1] for intv in intervals]

# number of equations used)
N = 4

# create Qr matrices
Q1 = (np.diag([-1 for x in range(0,N+1)],0) + np.diag([1 for x in range(0,N)],-1))[:-1, :]
Q2 = (np.diag([-x for x in range(0,N+1)],0) + np.diag([x for x in range(1,N+1)],1))[:-1, :]

# define bounds 
pl = np.array(pl)[:N + 1]
pu = np.array(pu)[:N + 1]

# Construct the problem.
k1 = cv.Variable(1)
k2 = 1
p = cv.Variable(N + 1)
z1 = cv.Variable(N + 1)
z2 = cv.Variable(N + 1)
objective_max = cv.Maximize(k1)
objective_min = cv.Minimize(k1)
constraints = [Q1 @ z1 + Q2 @ z2 == 0,
                k1 >= 0,
                p >= pl,
                p <= pu,
                z1 == k1 * p,
                z2 == k2 * p
                ]
prob_max = cv.Problem(objective_max, constraints)
prob_min = cv.Problem(objective_min, constraints)   

# Print result.
result_max = prob_max.solve()
print("\nThe upper bound is", prob_max.value)
print("A solution k1 is")
print(k1.value)
max_bound = k1.value
result_min = prob_min.solve()
print("\nThe lower bound is", prob_min.value)
print("A solution k1 is")
print(k1.value)
min_bound = k1.value

print(min_bound, max_bound)

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
var272 == Promote(var270, (5,)) @ var271 , because the following subexpressions are not:
|--  Promote(var270, (5,)) @ var271

# Scipy

### Example problem

max z = x + 2y \
s.t. 2x + y <= 20 \
     -4x + 5y <= 10 \
     -x + 2y >= -2 \
     -x + 5y = 15 \
     x >= 0 \
     y >= 0 

(note: convert constraints to >= 0, or = 0)

In [17]:
# Objective
obj = lambda x: x[0] + 2 * x[1]

# Constraints
cons = ({'type':'ineq', 'fun': lambda x: -2 * x[0] - x[1] + 20},
        {'type':'ineq', 'fun': lambda x: 4 * x[0] - 5 * x[1] + 10},
        {'type':'ineq', 'fun': lambda x: -x[0] + 2 * x[1] + 2},
        {'type':'eq', 'fun': lambda x: -x[0] + 5 * x[1] - 15})

# bounds
bnds = ((0, None), (0, None))

# solve
res = scipy.optimize.minimize(obj, (0,0), method="SLSQP", bounds=bnds,constraints=cons)

# print
print(res.x)

[1.66666667 3.33333333]


### Birth Death

x = [k1, p0, p1, ...], k2 = 1 fixed

Using non-linear constraints, as k1 * p involved in stationary condition

In [79]:
N = 3

# create confidence intervals
k1 = 2
x_max = 6
estimates, intervals = bootstrap([x for x in range(x_max)],0,[k1,1],100,1000,1000)

# create Qr matrices
Q1 = (np.diag([-1 for x in range(0,N+1)],0) + np.diag([1 for x in range(0,N)],-1))[:-1, :]
Q2 = (np.diag([-x for x in range(0,N+1)],0) + np.diag([x for x in range(1,N+1)],1))[:-1, :]

# create bounds on p
pl = [intv[0] for intv in intervals]
pu = [intv[1] for intv in intervals]

# define bounds 
pl = np.array(pl)[:N + 1]
pu = np.array(pu)[:N + 1]

# Objective
obj_min = lambda x: x[0]
obj_max = lambda x: -x[0]

# Constraints
cons = ({'type':'eq', 'fun': lambda x: x[0] * Q1 @ x[1:] + Q2 @ x[1:]})

# Bounds
bnds = ((0, None),) + tuple([(lb, ub) for lb, ub in zip(pl, pu)])

# initial value
x0 = tuple([0 for x in range(N + 2)])

# solve
res_min = scipy.optimize.minimize(obj_min, x0, method="SLSQP", bounds=bnds,constraints=cons)
res_max = scipy.optimize.minimize(obj_max, x0, method="SLSQP", bounds=bnds,constraints=cons)

# print
k1_lb = res_min.x[0]
k1_ub = res_max.x[0]
p_lb = res_min.x[1:]
p_ub = res_max.x[1:]
print(f"k1 in: ({k1_lb}, {k1_ub})")

k1 in: (1.8791592733200484, 2.2789744037360102)


Seems to perform very well, obtaining tight and accurate (95% CI bounds so sometimes off) bounds on k1

# Gene Expression Model

# Setup

In [80]:
def sample_path_gene(initial_state, params, tmax, plot=False):
    """
    Simulate a sample path of gene expression
    start at initial state, run until time tmax, rates in params
    return visited states and jump times
    """

    # initialise time and state (also store jump times)
    t = 0
    path = [(initial_state[0],initial_state[1])]
    jump_times = [0]
    k_on, k_off, k_tx, k_deg = params[0], params[1], params[2], params[3]

    # simulate until stopping time
    while t < tmax:
        # simulate holding time of current state (m,g)
        m, g = path[-1][0], path[-1][1]

        # define rates
        # q(x,x+(0,1)): gene turns on if off
        q_on = (1 - g) * k_on
        # q(x,x+(0,-1)): gene turns off if on
        q_off = g * k_off
        # q(x,x+(1,0)): transscription if gene on
        q_tx = g * k_tx
        # q(x,x+(-1,0)): degradation if  there are molecules
        q_deg = m * k_deg
        # -q(x,x): holding rate = sum q(x,y) over y
        q_hold = q_on + q_off + q_tx + q_deg

        # exponential holding time of state
        t_hold = -math.log(random.random()) / q_hold
        t += t_hold
        jump_times.append(t)

        # jump probability
        # P(x -> y) = q_xy / q_hold
        outcome = [1,2,3,4]
        prob = [q_on / q_hold, q_off / q_hold, q_tx / q_hold, q_deg / q_hold]
        jump = np.random.choice(outcome,p=prob)
        # jump to new state
        if jump == 1:
            path.append((m,g + 1))
        elif jump == 2:
            path.append((m,g - 1))
        elif jump == 3:
            path.append((m + 1,g))
        elif jump == 4:
            path.append((m - 1,g))

    if plot:
        # separate paths
        m_path = [state[0] for state in path]
        g_path = [state[1] for state in path]
        plt.scatter(jump_times, m_path, c=g_path, ec='k')

        # defined colour plotting
        #cmap, norm = mcolors.from_levels_and_colors([0, 1, 2], ['orange','blue'])
        #plt.scatter(jump_times, m_list, c=g_list, cmap=cmap, norm=norm)

    return path, jump_times

In [81]:
def bootstrap_marginal(m_list, initial_state, params, tmax, n, N, plot=False, printing=False):
    """
    m_list: marginal states of m to calculate probabilities for
    initial state
    parameters
    tmax: time of path before sample
    n = number of simulated samples
    N = number of bootstrap samples
    """

    # simulate n samples: (m,g)
    states = []
    for i in range(n):
        path, jumps = sample_path_gene(initial_state, params, tmax, plot=plot)
        states.append(path[-2]) # last state before tmax
    plt.show()

    # simulate N bootstrap samples: estimates p(m) for each, and for each m
    estimates = [[] for x in m_list]
    for i in range(N):
        # sample with replacement from (m,g) samples
        sample = random.choices(states,k = n)
        # take just marginal m information
        m_sample = [state[0] for state in sample]
        # estimate p(m) from marginal info
        for i, x in enumerate(m_list):
            estimates[i].append(m_sample.count(x) / n)

    # create confidence intervals (95%) via 2.5%, 97.5% quantiles for each x
    intervals = [np.quantile(est,[0.025,0.975]) for est in estimates]
    
    # plot histograms and CI
    for i, x in enumerate(m_list):
        if printing:
            print(f"95% CI for p({x}) is: ({intervals[i][0]}, {intervals[i][1]})")
        if plot:
            plt.hist(estimates[i])
            plt.title(f"Hist of p({x})")
            plt.axvline(intervals[i][0], color="red")
            plt.axvline(intervals[i][1], color="red")
            plt.show()

    # return CIs
    return estimates, intervals

# Non-linear program

x = [k1, k2, k3, k4, p(0,0), p(0,1), ... ]

i.e. x[4:] = p (joint probability vector)

In [101]:
# number of equations used
N = 4
# number of marginals used
M = int(N/2 + 1)
# parameters
k1, k2, k3, k4 = 1, 1 ,0.5, 1
# # number of marginal simulated (needs to be > M )
x_max = 10

# simulate data
states = [x for x in range(x_max)]
estimates_marginal, intervals_marginal = bootstrap_marginal(states,[0,0],[k1,k2,k3,k4],100,2000,2000,plot=False,printing=True)

# bounds from CI, on marginal
pl = [intr[0] for intr in intervals_marginal]
pu = [intr[1] for intr in intervals_marginal]

# define bounds
pl = np.array(pl)[:M]
pu = np.array(pu)[:M]

# marginal = A @ p
A = np.repeat(np.eye(M, dtype=int), repeats=2, axis=1)

# create Qr matrices
Q1 = (np.diag([-1 if not x % 2 else 0 for x in range(0,N+2)],0) +
        np.diag([1 if not x % 2 else 0 for x in range(0,N+1)],-1))[:-2, :]
Q2 = (np.diag([-1 if x % 2 else 0 for x in range(0,N+2)],0) +
        np.diag([1 if not x % 2 else 0 for x in range(0,N+1)],1))[:-2, :]
Q3 = (np.diag([-1 if x % 2 else 0 for x in range(0,N+2)],0) +
        np.diag([1 if x % 2 else 0 for x in range(0,N)],-2))[:-2, :]
Q4 = (np.diag([-1 if x > 1 else 0 for x in range(0,N+2)],0) +
        np.diag([1 for x in range(0,N)],2))[:-2, :]

# Objective (k3)
obj_min = lambda x: x[2]
obj_max = lambda x: -x[2]

# Constraints
cons = ({'type':'eq', 'fun': lambda x: x[0] * Q1 @ x[4:] + x[1] * Q2 @ x[4:] + x[2] * Q3 @ x[4:] + x[3] * Q4 @ x[4:]},
        {'type':'ineq', 'fun': lambda x: A @ x[4:] - pl},
        {'type':'ineq', 'fun': lambda x: pu - A @ x[4:]})

# Bounds: 4 rates, (N + 2) length p
bnds = ((0, None),(0, None),(0, None),(0, None)) + tuple([(0,1) for i in range(N + 2)])
#tuple([(lb, ub) for lb, ub in zip(pl, pu)]) (change to frechet bounds??)

# initial value: 4 rates, (N + 2) length p
x0 = tuple([0.5 for x in range(N + 6)])

# solve
res_min = scipy.optimize.minimize(obj_min, x0, method="SLSQP", bounds=bnds,constraints=cons)
res_max = scipy.optimize.minimize(obj_max, x0, method="SLSQP", bounds=bnds,constraints=cons)

# print
k3_lb = res_min.x[2]
k3_ub = res_max.x[2]
p_lb = res_min.x[4:]
p_ub = res_max.x[4:]
print(f"k3 in: ({k3_lb}, {k3_ub})")
print(res_min.x)
print(res_max.x)

95% CI for p(0) is: (0.762, 0.7985)
95% CI for p(1) is: (0.172, 0.2065)
95% CI for p(2) is: (0.0225, 0.037)
95% CI for p(3) is: (0.0, 0.0015)
95% CI for p(4) is: (0.0, 0.0015)
95% CI for p(5) is: (0.0, 0.0)
95% CI for p(6) is: (0.0, 0.0)
95% CI for p(7) is: (0.0, 0.0)
95% CI for p(8) is: (0.0, 0.0)
95% CI for p(9) is: (0.0, 0.0)
k3 in: (2.991085094991748e-21, 6.095956373721924e-11)
[3.89925809e-01 6.11704666e-01 2.99108509e-21 5.13513724e-21
 4.87650944e-01 3.10848845e-01 1.05042114e-01 6.69581802e-02
 2.82438201e-02 1.91657397e-04]
[4.06211005e-01 9.24662143e-01 6.09595637e-11 8.59856858e-11
 5.54780688e-01 2.43719312e-01 1.19501914e-01 5.24980859e-02
 8.18893513e-03 2.88110649e-02]
